In [1]:
import pandas as pd
from mlflow import MlflowClient
import mlflow
import os
from dotenv import load_dotenv

load_dotenv()
mlflow.set_tracking_uri(os.getenv('MLFLOW_TRACKING_URI'))
username = os.getenv('MLFLOW_TRACKING_USERNAME')
password = os.getenv('MLFLOW_TRACKING_PASSWORD')

mlflow_client = MlflowClient()

In [2]:
city = 'MARIBOR'
model_name = f"{city}_model"
latest_model_version = mlflow_client.get_latest_versions(model_name)
latest_model = mlflow_client.get_model_version(model_name, latest_model_version[-1].version)
model_uri = f'runs:/{latest_model.run_id}/model'
model = mlflow.pyfunc.load_model(model_uri)

C:\Users\Tine Šuster\AppData\Local\Temp\ipykernel_20568\2804614595.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_model_version = mlflow_client.get_latest_versions(model_name)


In [3]:
import numpy as np


def create_windows(df, window_size, prediction_horizon, target_column='european_aqi'):
    data = df.to_numpy()
    target_index = df.columns.get_loc(target_column)

    X, y = [], []
    for i in range(len(data) - window_size - prediction_horizon + 1):
        X.append(data[i:i + window_size, :])
        y.append(data[i + window_size:i + window_size + prediction_horizon, target_index])
    return np.array(X), np.array(y)


In [16]:
data = pd.read_csv(f'../data/processed/{city}.csv')
data.drop(columns=['time'], inplace=True)
data = np.expand_dims(data.values, axis=0)
X, y = create_windows(data, 24, 1)

AttributeError: 'numpy.ndarray' object has no attribute 'to_numpy'

In [12]:
import shap

explainer = shap.KernelExplainer(model.predict, X[0:100])
shape_values = explainer.shap_values(X[0:100])

Provided model function fails when applied to the provided data set.


MlflowException: Failed to enforce schema of data '[[[23.7 58.  14.9 ...  0.2 67.  27. ]
  [23.2 61.  15.2 ...  0.2 66.  26. ]
  [22.3 66.  15.6 ...  0.2 69.  28. ]
  ...
  [22.9 58.  14.2 ...  0.3 74.  30. ]
  [23.9 47.  11.9 ...  0.2 74.  30. ]
  [22.8 57.  13.8 ...  0.2 71.  28. ]]

 [[23.2 61.  15.2 ...  0.2 66.  26. ]
  [22.3 66.  15.6 ...  0.2 69.  28. ]
  [21.9 74.  17.  ...  0.2 70.  28. ]
  ...
  [23.9 47.  11.9 ...  0.2 74.  30. ]
  [22.8 57.  13.8 ...  0.2 71.  28. ]
  [23.1 60.  14.9 ...  0.2 70.  28. ]]

 [[22.3 66.  15.6 ...  0.2 69.  28. ]
  [21.9 74.  17.  ...  0.2 70.  28. ]
  [21.3 78.  17.3 ...  0.2 67.  27. ]
  ...
  [22.8 57.  13.8 ...  0.2 71.  28. ]
  [23.1 60.  14.9 ...  0.2 70.  28. ]
  [22.8 68.  16.6 ...  0.2 64.  26. ]]

 ...

 [[15.3 94.  14.4 ...  1.7 34.  12. ]
  [15.2 94.  14.2 ...  1.7 31.  16. ]
  [15.  95.  14.2 ...  1.7 32.  16. ]
  ...
  [16.  85.  13.5 ...  0.9 48.  19. ]
  [15.4 88.  13.4 ...  0.9 44.  18. ]
  [14.9 90.  13.3 ...  0.2 53.  21. ]]
